In [22]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
from pandas_datareader import data as pdr

# 1) Settings
start, end = "2020-01-01", "2025-06-30"
india_tickers = ["SHEMAROO.NS", "INNOVANA.NS"]
us_tickers    = ["AVAV", "HRTX"]
all_tickers   = india_tickers + us_tickers + ["^GSPC"]

# 2) Download Close prices
prices = yf.download(all_tickers, start=start, end=end)["Close"]

# 3) Download risk‑free rate and convert to daily
rf = pdr.DataReader("DTB3", "fred", start, end).dropna() / 100.0
rf_daily = rf.reindex(prices.index).ffill() / 252.0

# 4) Compute daily returns
rets       = prices.pct_change().dropna()
market_ret = rets["^GSPC"]
rf_ser     = rf_daily["DTB3"]

# 5) CAPM + LPQ performance function
def capm_performance(y, mkt, rf):
    # Excess returns
    Y = y - rf
    X = pd.DataFrame({"mkt_exc": mkt - rf})
    X = sm.add_constant(X)   # columns: ["const","mkt_exc"]
    
    # OLS fit
    ols = sm.OLS(Y, X).fit()
    params = ols.params              # Series indexed ["const","mkt_exc"]
    yhat_ols = ols.predict(X)
    res_ols  = Y - yhat_ols
    
    # LPQ correction
    res = res_ols.values
    n, p = X.shape
    sigma2 = np.var(res, ddof=p)
    m3     = np.mean((res - res.mean())**3) / sigma2**1.5
    m4     = np.mean((res - res.mean())**4) / sigma2**2
    exk    = m4 - 3.0
    gamma  = m3 / (2.0 + exk) if (2.0 + exk)!=0 else 0.0
    
    corr = (gamma * (X.values.T @ (res**2))) / n
    lpq_params = params.values + corr      # numpy array length 2
    yhat_lpq   = X.values @ lpq_params
    res_lpq    = Y - yhat_lpq
    
    # Performance metrics
    mse_ols = np.mean(res_ols**2)
    mse_lpq = np.mean(res_lpq**2)
    mae_ols = np.mean(np.abs(res_ols))
    mae_lpq = np.mean(np.abs(res_lpq))
    
    return {
        "OLS α":    params["const"],
        "OLS β":    params["mkt_exc"],
        "LPQ α":    lpq_params[0],
        "LPQ β":    lpq_params[1],
        "MSE_OLS":  mse_ols,
        "MSE_LPQ":  mse_lpq,
        "MAE_OLS":  mae_ols,
        "MAE_LPQ":  mae_lpq
    }

# 6) Loop tickers and collect performance
rows = []
for ticker in india_tickers + us_tickers:
    df = pd.concat([rets[ticker], market_ret, rf_ser], axis=1).dropna()
    df.columns = ["ret","mkt","rf"]
    perf = capm_performance(df["ret"], df["mkt"], df["rf"])
    perf["Ticker"] = ticker
    rows.append(perf)

# 7) Display comparison table
perf_df = pd.DataFrame(rows).set_index("Ticker")
print(perf_df)


/var/folders/hx/ckgg6j1s22l73jpkp3g7np640000gn/T/ipykernel_46487/3169567072.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(all_tickers, start=start, end=end)["Close"]
[*********************100%***********************]  5 of 5 completed


                OLS α     OLS β     LPQ α     LPQ β   MSE_OLS   MSE_LPQ  \
Ticker                                                                    
SHEMAROO.NS  0.000342  0.032037  0.000521  0.032037  0.001381  0.001381   
INNOVANA.NS  0.001999  0.009745  0.002162  0.009745  0.001190  0.001190   
AVAV         0.001047  1.000629  0.001094  1.000629  0.000922  0.000922   
HRTX        -0.000923  1.305421 -0.000776  1.305421  0.002573  0.002573   

              MAE_OLS   MAE_LPQ  
Ticker                           
SHEMAROO.NS  0.025559  0.025584  
INNOVANA.NS  0.024531  0.024581  
AVAV         0.018192  0.018196  
HRTX         0.033502  0.033506  


/var/folders/hx/ckgg6j1s22l73jpkp3g7np640000gn/T/ipykernel_46487/3169567072.py:21: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets       = prices.pct_change().dropna()
